In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from tqdm.auto import tqdm

from numpy.random import seed
seed(42)
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras
from keras import backend as K

from sklearn.model_selection import KFold, GroupKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder, StandardScaler
from sklearn.preprocessing import QuantileTransformer

def root_mean_squared_per_error(y_true, y_pred):
         return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true)))
    
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0,
    mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=7, verbose=0,
    mode='min')

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
train = pd.read_pickle('./input/train_248.pkl')
test = pd.read_pickle('./input/test_247.pkl')

In [4]:
colNames = [col for col in list(train.columns)
            if col not in {"stock_id", "time_id", "target", "row_id"}]
len(colNames)

244

In [5]:
#colNames.remove('row_id')
# train.replace([np.inf, -np.inf], np.nan,inplace=True)
# test.replace([np.inf, -np.inf], np.nan,inplace=True)
qt_train = []
train_nn=train[colNames].copy()
test_nn=test[colNames].copy()
# 每次对一列做变换，节约内存，且输出数据格式不全为float64，节省存储空间
for col in tqdm(colNames, total=len(colNames)):
    #print(col)
    qt = QuantileTransformer(random_state=21,n_quantiles=2000, output_distribution='normal')  # 将每个特征缩放在同样的范围或分布下。通过执行一个秩转换能够使异常的分布平滑化，并且能够比缩放更少的收到离群值的影响。
    train_nn[col] = qt.fit_transform(train_nn[[col]])
    test_nn[col] = qt.transform(test_nn[[col]])    
    qt_train.append(qt)

In [6]:
# 一次性做变换，内存占用大，输出的每列数据全为float64（统一）
# train_nn_2=train[colNames].copy()
# test_nn_2=test[colNames].copy()
# qt = QuantileTransformer(random_state=21,n_quantiles=2000, output_distribution='normal')  # 将每个特征缩放在同样的范围或分布下。通过执行一个秩转换能够使异常的分布平滑化，并且能够比缩放更少的收到离群值的影响。
# train_nn_2[colNames] = qt.fit_transform(train_nn_2[colNames])
# test_nn_2[colNames] = qt.transform(test_nn[colNames])

In [7]:
# train.replace([np.inf, -np.inf], np.nan,inplace=True)
# test.replace([np.inf, -np.inf], np.nan,inplace=True)

train_nn[['stock_id','time_id','target']] = train[['stock_id','time_id','target']]
test_nn[['stock_id','time_id']] = test[['stock_id','time_id']]

In [8]:
train_nn

,wap1_sum,wap1_std,wap2_sum,wap2_std,wap3_sum,wap3_std,wap4_sum,wap4_std,log_return1_realized_volatility,log_return2_realized_volatility,log_return3_realized_volatility,log_return4_realized_volatility,wap_balance_sum,wap_balance_amax,price_spread_sum,price_spread_amax,price_spread2_sum,price_spread2_amax,bid_spread_sum,bid_spread_amax,ask_spread_sum,ask_spread_amax,total_volume_sum,total_volume_amax,volume_imbalance_sum,volume_imbalance_amax,bid_ask_spread_sum,bid_ask_spread_amax,log_return1_realized_volatility_500,log_return2_realized_volatility_500,log_return3_realized_volatility_500,log_return4_realized_volatility_500,log_return1_realized_volatility_400,log_return2_realized_volatility_400,log_return3_realized_volatility_400,log_return4_realized_volatility_400,log_return1_realized_volatility_300,log_return2_realized_volatility_300,log_return3_realized_volatility_300,log_return4_realized_volatility_300,log_return1_realized_volatility_200,log_return2_realized_volatility_200,log_return3_realized_volatility_200,log_return4_realized_volatility_200,log_return1_realized_volatility_100,log_return2_realized_volatility_100,log_return3_realized_volatility_100,log_return4_realized_volatility_100,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_size_amax,trade_size_amin,trade_order_count_sum,trade_order_count_amax,trade_amount_sum,trade_amount_amax,trade_amount_amin,trade_tendency,trade_f_max,trade_f_min,trade_df_max,trade_df_min,trade_abs_diff,trade_energy,trade_iqr_p,trade_abs_diff_v,trade_energy_v,trade_iqr_p_v,trade_log_return_realized_volatility_500,trade_seconds_in_bucket_count_unique_500,trade_size_sum_500,trade_order_count_sum_500,trade_log_return_realized_volatility_400,trade_seconds_in_bucket_count_unique_400,trade_size_sum_400,trade_order_count_sum_400,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_sum_300,trade_log_return_realized_volatility_200,trade_seconds_in_bucket_count_unique_200,trade_size_sum_200,trade_order_count_sum_200,trade_log_return_realized_volatility_100,trade_seconds_in_bucket_count_unique_100,trade_size_sum_100,trade_order_count_sum_100,size_tau,size_tau2,size_tau_200,size_tau2_200,size_tau_300,size_tau2_300,size_tau_400,size_tau2_400,size_tau2_d,log_return1_realized_volatility_mean_stock,log_return1_realized_volatility_std_stock,log_return1_realized_volatility_max_stock,log_return1_realized_volatility_min_stock,log_return2_realized_volatility_mean_stock,log_return2_realized_volatility_std_stock,log_return2_realized_volatility_max_stock,log_return2_realized_volatility_min_stock,log_return1_realized_volatility_400_mean_stock,log_return1_realized_volatility_400_std_stock,log_return1_realized_volatility_400_max_stock,log_return1_realized_volatility_400_min_stock,log_return2_realized_volatility_400_mean_stock,log_return2_realized_volatility_400_std_stock,log_return2_realized_volatility_400_max_stock,log_return2_realized_volatility_400_min_stock,log_return1_realized_volatility_300_mean_stock,log_return1_realized_volatility_300_std_stock,log_return1_realized_volatility_300_max_stock,log_return1_realized_volatility_300_min_stock,log_return2_realized_volatility_300_mean_stock,log_return2_realized_volatility_300_std_stock,log_return2_realized_volatility_300_max_stock,log_return2_realized_volatility_300_min_stock,log_return1_realized_volatility_200_mean_stock,log_return1_realized_volatility_200_std_stock,log_return1_realized_volatility_200_max_stock,log_return1_realized_volatility_200_min_stock,log_return2_realized_volatility_200_mean_stock,log_return2_realized_volatility_200_std_stock,log_return2_realized_volatility_200_max_stock,log_return2_realized_volatility_200_min_stock,trade_log_return_realized_volatility_mean_stock,trade_log_return_realized_volatility_std_stock,trade_log_return_realized_volatility_max_stock,trade_log_return_realized_volatility_min_stock,trade_log_return_realized_volatility_400_

In [9]:
#https://bignerdranch.com/blog/implementing-swish-activation-function-in-keras/
from keras.backend import sigmoid,tanh,log,exp
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

def swish_2(x, beta = 0.95):
    return (x * sigmoid(beta * x))

def swish_3(x, beta = 1.05):
    return (x * sigmoid(beta * x))

def mish(x):
    return (x * tanh(log(1+exp(x))))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})
# get_custom_objects().update({'mish': Activation(mish)})

# hidden_units = (128,64,32)
# hidden_units = (512,128,32)
# hidden_units = (256,128,64,16)
# hidden_units = (512,128,64,16)
stock_embedding_size = 24

cat_data = train_nn['stock_id']

def base_model():
    
    # Each instance will consist of two inputs: a single user id, and a single movie id
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(244,), name='num_data') # 247 - 1(stock_id) - 2(time_id, target) =244  # 258


    #embedding, flatenning and concatenating
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_input])
    
    # Add one or more hidden layers
    hidden_units = (128,64,32)
    for n_hidden in hidden_units:
        out = keras.layers.Dense(n_hidden, activation='swish')(out)
#         out = keras.layers.Dense(n_hidden, activation='mish')(out)
    #out = keras.layers.Concatenate()([out, num_input])

    # A single output: our predicted rating
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out,
    )
    
    return model


def base_model_2(num_columns,  # 244
#                num_labels,  # output_dim
               hidden_units,   # (128,64,32)
               dropout_rates,  # list of dropout_rates
               stock_embedding_size = 24, 
               # ls = 1e-2,  # label_smoothing = ls
               lr = 1e-3):
    
    # input 
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(num_columns,), name='num_data')
    
    #embedding, flatenning and concatenating
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, 
                                            stock_embedding_size,
                                            input_length=1, 
                                            name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    x = keras.layers.Concatenate()([stock_flattened, num_input])
    
    i=0
    # Add one or more hidden layers
    for n_hidden in hidden_units:
#         out = keras.layers.Dense(n_hidden, activation='swish')(out)
        x = keras.layers.Dense(n_hidden)(x)
#         if i%2 ==1:
#             x = keras.layers.BatchNormalization()(x)
        if i==0 or i==4:
            x = tf.keras.layers.GaussianNoise(0.1)(x)  # std of noise
        x = keras.layers.Activation('swish')(x)
#         x = keras.layers.Activation('swish')(x)
        x = keras.layers.Dropout(dropout_rates[i])(x)
        i+=1

    out = keras.layers.Dense(1, activation='linear', name='prediction')(x)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out,
    )
    
    return model

In [10]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [11]:
%%time
target_name='target'
scores_folds = {}
model_name = 'NN'
pred_name = 'pred_{}'.format(model_name)

n_folds = 5
# kf = KFold(n_splits=n_folds, shuffle=True, random_state=2020)
kf = GroupKFold(n_splits=5)
scores_folds[model_name] = []

features_to_consider = list(train_nn)  # len -> 247
features_to_consider.remove('time_id')
features_to_consider.remove('target')
# features_to_consider = [col for col in train_nn.columns if col not in {"time_id", "target"}]
try:
    features_to_consider.remove('pred_NN')
except:
    pass

train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
train_nn[pred_name] = 0
test_nn[target_name] = 0
test_predictions_nn1 = np.zeros(test_nn.shape[0])
valid_predictions_nn1 = np.zeros(train_nn.shape[0])

# for fold, (trn_ind, val_ind) in enumerate(kf.split(train)):
for fold, (trn_ind, val_ind) in enumerate(kf.split(train, groups=train['time_id'])):
    print(f'Training fold {fold + 1}')
    
    X_train = train_nn.loc[trn_ind, features_to_consider]
    y_train = train_nn.loc[trn_ind, target_name]
    X_test = train_nn.loc[val_ind, features_to_consider]
    y_test = train_nn.loc[val_ind, target_name]

    model = base_model()
    
    model.compile(
        keras.optimizers.Adam(learning_rate=0.006),
#         keras.optimizers.Adam(learning_rate=0.006,decay=3e-5),
        loss=root_mean_squared_per_error
    )
    try:
        features_to_consider.remove('stock_id')
    except:
        pass
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    num_data = X_train[features_to_consider]
    num_data = scaler.fit_transform(num_data.values)
    cat_data = X_train['stock_id']
    
    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']
    
    
    model.fit([cat_data, num_data],
              y_train, 
              batch_size=2048, 
              epochs=1000, 
#               sample_weight= 1/np.square(y_train),
              validation_data=([cat_data_test, num_data_test], y_test), 
              callbacks=[es, plateau], 
              validation_batch_size=len(y_test), 
              shuffle=True, 
              verbose = 1)
    
    # predict valid
    valid_predictions_nn1[val_ind] = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
#     score = round(rmspe(y_true = y_test, y_pred = preds),5)
    score = round(rmspe(y_true = y_test, y_pred = valid_predictions_nn1[val_ind]),5)
    print('Fold {} {}: {}'.format(fold+1, model_name, score))
    scores_folds[model_name].append(score)
    
    # predict test
    tt =scaler.transform(test_nn[features_to_consider].values)
    test_predictions_nn1 += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)/n_folds
    features_to_consider.append('stock_id')
    
print('avg val loss: ', round(np.mean(scores_folds['NN']), 5))

Training fold 1
Epoch 1/1000
168/168 [==============================] - 3s 11ms/step - loss: 6.3244 - val_loss: 0.4114
Epoch 2/1000
168/168 [==============================] - 2s 11ms/step - loss: 0.5647 - val_loss: 0.5126
Epoch 3/1000
168/168 [==============================] - 2s 10ms/step - loss: 0.7246 - val_loss: 0.6406
Epoch 4/1000
168/168 [==============================] - 2s 10ms/step - loss: 0.6388 - val_loss: 0.6318
Epoch 5/1000
168/168 [==============================] - 2s 11ms/step - loss: 0.6431 - val_loss: 0.4904
Epoch 6/1000
168/168 [==============================] - 2s 13ms/step - loss: 0.6300 - val_loss: 0.7477
Epoch 7/1000
168/168 [==============================] - 2s 12ms/step - loss: 0.4966 - val_loss: 0.6175
Epoch 8/1000
168/168 [==============================] - 2s 13ms/step - loss: 0.6613 - val_loss: 0.3345
Epoch 9/1000
168/168 [==============================] - 2s 13ms/step - loss: 0.4154 - val_loss: 0.4375
Epoch 10/1000
168/168 [==============================] - 

Epoch 27/1000
168/168 [==============================] - 1s 8ms/step - loss: 0.2317 - val_loss: 0.2303
Epoch 28/1000
168/168 [==============================] - 1s 8ms/step - loss: 0.2271 - val_loss: 0.2574
Epoch 29/1000
168/168 [==============================] - 1s 8ms/step - loss: 0.2303 - val_loss: 0.2697
Epoch 30/1000
168/168 [==============================] - 1s 8ms/step - loss: 0.2257 - val_loss: 0.2432
Epoch 31/1000
168/168 [==============================] - 1s 8ms/step - loss: 0.2362 - val_loss: 0.2385
Epoch 32/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2641 - val_loss: 0.2253
Epoch 33/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2617 - val_loss: 0.2666
Epoch 34/1000
168/168 [==============================] - 1s 7ms/step - loss: 0.2317 - val_loss: 0.2287
Epoch 35/1000
168/168 [==============================] - 1s 8ms/step - loss: 0.2205 - val_loss: 0.2235
Epoch 36/1000
168/168 [==============================] - 1s 8ms/step - lo

168/168 [==============================] - 1s 9ms/step - loss: 0.2071 - val_loss: 0.2180
Epoch 40/1000
168/168 [==============================] - 2s 9ms/step - loss: 0.2070 - val_loss: 0.2114
Epoch 41/1000
168/168 [==============================] - 1s 9ms/step - loss: 0.2035 - val_loss: 0.2131
Epoch 42/1000
168/168 [==============================] - 2s 9ms/step - loss: 0.2032 - val_loss: 0.2105
Epoch 43/1000
168/168 [==============================] - 2s 9ms/step - loss: 0.2032 - val_loss: 0.2092
Epoch 44/1000
168/168 [==============================] - 2s 9ms/step - loss: 0.2031 - val_loss: 0.2110
Epoch 45/1000
168/168 [==============================] - 2s 9ms/step - loss: 0.2029 - val_loss: 0.2098
Epoch 46/1000
168/168 [==============================] - 1s 9ms/step - loss: 0.2028 - val_loss: 0.2102
Epoch 47/1000
168/168 [==============================] - 2s 9ms/step - loss: 0.2031 - val_loss: 0.2095
Epoch 48/1000
168/168 [==============================] - 2s 9ms/step - loss: 0.2030 - v

168/168 [==============================] - 1s 9ms/step - loss: 0.2045 - val_loss: 0.2111
Epoch 56/1000
168/168 [==============================] - 1s 9ms/step - loss: 0.2048 - val_loss: 0.2122
Epoch 57/1000
168/168 [==============================] - 1s 9ms/step - loss: 0.2035 - val_loss: 0.2097
Epoch 58/1000
168/168 [==============================] - 1s 9ms/step - loss: 0.2035 - val_loss: 0.2100
Epoch 59/1000
168/168 [==============================] - 2s 9ms/step - loss: 0.2034 - val_loss: 0.2100
Epoch 60/1000
168/168 [==============================] - 2s 9ms/step - loss: 0.2033 - val_loss: 0.2095
Epoch 61/1000
168/168 [==============================] - 2s 9ms/step - loss: 0.2033 - val_loss: 0.2099
Epoch 62/1000
168/168 [==============================] - 2s 9ms/step - loss: 0.2034 - val_loss: 0.2100
Epoch 63/1000
168/168 [==============================] - 2s 9ms/step - loss: 0.2033 - val_loss: 0.2098
Epoch 64/1000
168/168 [==============================] - 1s 9ms/step - loss: 0.2032 - v

168/168 [==============================] - 2s 10ms/step - loss: 0.2019 - val_loss: 0.2101
Epoch 55/1000
168/168 [==============================] - 1s 9ms/step - loss: 0.2019 - val_loss: 0.2101
Epoch 56/1000
168/168 [==============================] - 1s 9ms/step - loss: 0.2019 - val_loss: 0.2096
Fold 5 NN: 0.20924
avg val loss:  0.2127
Wall time: 8min 31s


In [12]:
%%time
target_name='target'
scores_folds = {}
model_name = 'NN2'
pred_name = 'pred_{}'.format(model_name)

n_folds = 5
# kf = KFold(n_splits=n_folds, shuffle=True, random_state=2020)
kf = GroupKFold(n_splits=5)
scores_folds[model_name] = []

features_to_consider = list(train_nn)  # len -> 247
features_to_consider.remove('time_id')
features_to_consider.remove('target')
# features_to_consider = [col for col in train_nn.columns if col not in {"time_id", "target"}]
try:
    features_to_consider.remove('pred_NN')
except:
    pass

train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
train_nn[pred_name] = 0
test_nn[target_name] = 0
test_predictions_nn2 = np.zeros(test_nn.shape[0])
valid_predictions_nn2 = np.zeros(train_nn.shape[0])

# for fold, (trn_ind, val_ind) in enumerate(kf.split(train)):
for fold, (trn_ind, val_ind) in enumerate(kf.split(train, groups=train['time_id'])):
    print(f'Training fold {fold + 1}')
    
    X_train = train_nn.loc[trn_ind, features_to_consider]
    y_train = train_nn.loc[trn_ind, target_name]
    X_test = train_nn.loc[val_ind, features_to_consider]
    y_test = train_nn.loc[val_ind, target_name]

    model = base_model_2(244,
                       hidden_units=[512,256,128,128,64,64,32,16],
                       # hidden_units=[128,128,128,128,32],   #[256,128,64]
                       dropout_rates=[0.02,0.02,0.01,0.01,0.01,0.02,0.02,0.02]
                       # dropout_rates=[0.2,0.2,0.2,0.2,0.2,0.2,]
                      )
    
    model.compile(
        keras.optimizers.Adam(learning_rate=0.006),
#         keras.optimizers.Adam(learning_rate=0.006,decay=3e-5),
        loss=root_mean_squared_per_error
    )
    try:
        features_to_consider.remove('stock_id')
    except:
        pass
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    num_data = X_train[features_to_consider]
    num_data = scaler.fit_transform(num_data.values)
    cat_data = X_train['stock_id']
    
    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']
    
    
    model.fit([cat_data, num_data],
              y_train, 
              batch_size=2048, 
              epochs=1000, 
#               sample_weight= 1/np.square(y_train),
              validation_data=([cat_data_test, num_data_test], y_test), 
              callbacks=[es, plateau], 
              validation_batch_size=len(y_test), 
              shuffle=True, 
              verbose = 1)
    
    # predict valid
    valid_predictions_nn2[val_ind] = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
#     score = round(rmspe(y_true = y_test, y_pred = preds),5)
    score = round(rmspe(y_true = y_test, y_pred = valid_predictions_nn1[val_ind]),5)
    print('Fold {} {}: {}'.format(fold+1, model_name, score))
    scores_folds[model_name].append(score)
    
    # predict test
    tt =scaler.transform(test_nn[features_to_consider].values)
    test_predictions_nn2 += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)/n_folds
    features_to_consider.append('stock_id')
    
print('avg val loss: ', round(np.mean(scores_folds['NN2']), 5))

Training fold 1
Epoch 1/1000
168/168 [==============================] - 13s 72ms/step - loss: 1.7808 - val_loss: 0.4553
Epoch 2/1000
168/168 [==============================] - 12s 72ms/step - loss: 0.5697 - val_loss: 0.4714
Epoch 3/1000
168/168 [==============================] - 12s 72ms/step - loss: 0.3996 - val_loss: 0.3541
Epoch 4/1000
168/168 [==============================] - 12s 73ms/step - loss: 0.3197 - val_loss: 0.3147
Epoch 5/1000
168/168 [==============================] - 12s 73ms/step - loss: 0.2750 - val_loss: 0.2265
Epoch 6/1000
168/168 [==============================] - 12s 73ms/step - loss: 0.2654 - val_loss: 0.3131
Epoch 7/1000
168/168 [==============================] - 12s 73ms/step - loss: 0.2928 - val_loss: 0.2251
Epoch 8/1000
168/168 [==============================] - 12s 74ms/step - loss: 0.2368 - val_loss: 0.2266
Epoch 9/1000
168/168 [==============================] - 12s 73ms/step - loss: 0.2262 - val_loss: 0.2655
Epoch 10/1000
168/168 [=========================

168/168 [==============================] - 11s 66ms/step - loss: 0.5479 - val_loss: 0.2936
Epoch 3/1000
168/168 [==============================] - 11s 68ms/step - loss: 0.3547 - val_loss: 0.5517
Epoch 4/1000
168/168 [==============================] - 14s 81ms/step - loss: 0.3630 - val_loss: 0.2522
Epoch 5/1000
168/168 [==============================] - 13s 75ms/step - loss: 0.2815 - val_loss: 0.4316
Epoch 6/1000
168/168 [==============================] - 11s 64ms/step - loss: 0.2893 - val_loss: 0.2346
Epoch 7/1000
168/168 [==============================] - 11s 67ms/step - loss: 0.2361 - val_loss: 0.2336
Epoch 8/1000
168/168 [==============================] - 11s 65ms/step - loss: 0.2328 - val_loss: 0.2349
Epoch 9/1000
168/168 [==============================] - 11s 66ms/step - loss: 0.2377 - val_loss: 0.2421
Epoch 10/1000
168/168 [==============================] - 11s 63ms/step - loss: 0.2275 - val_loss: 0.2305
Epoch 11/1000
168/168 [==============================] - 11s 63ms/step - los

Epoch 38/1000
168/168 [==============================] - 12s 70ms/step - loss: 0.2070 - val_loss: 0.2074
Epoch 39/1000
168/168 [==============================] - 12s 72ms/step - loss: 0.2066 - val_loss: 0.2182
Epoch 40/1000
168/168 [==============================] - 12s 72ms/step - loss: 0.2090 - val_loss: 0.2215
Epoch 41/1000
168/168 [==============================] - 12s 72ms/step - loss: 0.2062 - val_loss: 0.2113
Epoch 42/1000
168/168 [==============================] - 10s 62ms/step - loss: 0.2069 - val_loss: 0.2142
Epoch 43/1000
168/168 [==============================] - 11s 63ms/step - loss: 0.2087 - val_loss: 0.2120
Epoch 44/1000
168/168 [==============================] - 11s 65ms/step - loss: 0.2055 - val_loss: 0.2097
Epoch 45/1000
168/168 [==============================] - 11s 67ms/step - loss: 0.2066 - val_loss: 0.2127
Epoch 46/1000
168/168 [==============================] - 11s 68ms/step - loss: 0.2015 - val_loss: 0.2102
Epoch 47/1000
168/168 [==============================] 

168/168 [==============================] - 14s 73ms/step - loss: 1.6746 - val_loss: 0.4663
Epoch 2/1000
168/168 [==============================] - 12s 74ms/step - loss: 0.4231 - val_loss: 0.2335
Epoch 3/1000
168/168 [==============================] - 12s 69ms/step - loss: 0.4142 - val_loss: 0.4714
Epoch 4/1000
168/168 [==============================] - 10s 61ms/step - loss: 0.4324 - val_loss: 0.2371
Epoch 5/1000
168/168 [==============================] - 11s 65ms/step - loss: 0.3413 - val_loss: 0.2208
Epoch 6/1000
168/168 [==============================] - 12s 72ms/step - loss: 0.2594 - val_loss: 0.2299
Epoch 7/1000
168/168 [==============================] - 12s 72ms/step - loss: 0.2414 - val_loss: 0.2204
Epoch 8/1000
168/168 [==============================] - 12s 72ms/step - loss: 0.2387 - val_loss: 0.2209
Epoch 9/1000
168/168 [==============================] - 12s 73ms/step - loss: 0.2394 - val_loss: 0.2318
Epoch 10/1000
168/168 [==============================] - 12s 73ms/step - loss

In [13]:
valid_predictions_nn1, valid_predictions_nn2

(array([0.00366505, 0.0014547 , 0.00219503, ..., 0.00327135, 0.00471491,
        0.00199602]),
 array([0.00370796, 0.00146185, 0.0021736 , ..., 0.00322349, 0.00478484,
        0.0019784 ]))

In [14]:
test_predictions_nn1, test_predictions_nn2

(array([0.00225556, 0.00245739, 0.00245739]),
 array([0.00178146, 0.00283218, 0.00283218]))

In [15]:
pd.Series(valid_predictions_nn1).to_csv('outputs/nn1_valid_pred.csv',index=0)
pd.Series(valid_predictions_nn2).to_csv('outputs/nn2_valid_pred.csv',index=0)

In [16]:
import gc
del train_nn, test_nn
del X_train, X_test
del cat_data, num_data, cat_data_test, num_data_test
gc.collect()

9343